# Metadata

**L1 Taxonomy** - Software Architecture & Design

**L2 Taxonomy** - Observer Pattern

**Subtopic** - Implement the Observer Pattern to Monitor Weather Data

**Use Case** - Develop a Python program that implements the Observer Pattern to monitor weather data. The program should include a 'WeatherData' subject that tracks changes in temperature, humidity, and pressure. The program should also include 'CurrentConditionsDisplay', 'StatisticsDisplay', and 'ForecastDisplay' observers that update whenever 'WeatherData' changes. The observers should print the updated weather conditions whenever they receive an update from 'WeatherData'.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt


**Problem Description**
You must implement a thread-safe, production-grade Observer-pattern system in Python to monitor weather data. A central `WeatherData` subject tracks three metrics temperature, humidity, and pressure and notifies registered observers whenever these measurements change. Observers may be plain functions or objects implementing an `update(temp, humidity, pressure)` method. Notifications must support priority ordering, one-shot delivery, TTL expiry, and optional filtering, while remaining robust under concurrent updates and observer failures.

**Input Format and Constraints**

* No direct stdin input.
* All interactions occur via method calls on a `WeatherData` instance.
* Temperatures, humidity, and pressure are `float` values; you may assume callers pass valid floats.

**Expected Output Format**

* No return value from updates; observers print to console.
* `get_metrics()` returns a `Dict[int, float]` mapping subscription tokens to their average notification latency.
* `get_log()` returns a `List[str]` of timestamped event strings.

**Examples**

```python
wd = WeatherData()
# Observer objects
class Display:
    def update(self, t,h,p): print(f"Got {t},{h},{p}")
disp = Display()
token1 = wd.subscribe(disp, priority=10)
token2 = wd.subscribe(lambda t,h,p: print("Fn",t), once=True)
wd.set_measurements(22.5, 55.0, 1012.3)
# Console:
#   Got 22.5,55.0,1012.3
#   Fn 22.5
# Second call only notifies disp
wd.set_measurements(23.0, 60.0, 1011.8)
```



# Requirements



**Requirements**

1. **Explicit & Implicit Points**

   * Use the Observer pattern: one `WeatherData` subject, many observers.
   * Observers may be:

     * Any callable `(float, float, float) -> None`, wrapped automatically.
     * Any object with an `update(temp: float, humidity: float, pressure: float) -> None` method.
   * Must handle concurrent `subscribe`, `unsubscribe`, and `set_measurements` calls safely.

2. **Solution Expectations**

   * Notifications always go to all eligible observers in correct priority order, even if some raise exceptions.
   * Automatic unsubscription after one notification (`once=True`) or after N notifications (`ttl=N`).
   * Optional `filter_fn` predicate on the tuple `(temp, humidity, pressure)`.
   * Dead (garbage-collected) observers are removed on next notification.
   * Ability to pause/resume notifications.
   * Collection of per-observer timing metrics and an event log of every state change.

3. **Function Signatures**

   ```python
   class WeatherData:
       def subscribe(
           self,
           callback: Callable[[float, float, float], None],
           *,
           priority: int = 100,
           once: bool = False,
           ttl: Optional[int] = None,
           filter_fn: Optional[Callable[[Tuple[float, float, float]], bool]] = None
       ) -> int: ...

       def unsubscribe(self, token: int) -> None: ...
       def pause(self) -> None: ...
       def resume(self) -> None: ...
       def set_measurements(self, temp: float, humidity: float, pressure: float) -> None: ...
       def get_metrics(self) -> Dict[int, float]: ...
       def get_log(self) -> List[str]: ...
   ```

4. **Class & Data Structure Definitions**

   * `_ObserverEntry`: internal class with fields

     * `priority: int`, `index: int`, `token: int`, `once: bool`, `ttl: Optional[int]`,
     * `filter_fn: Optional[Callable]`, `ref: weakref.ReferenceType`
     * `__lt__` orders by `(priority, index)`.
   * Observers stored in a `heapq` min-heap for O(log N) insertion & removal.

5. **Edge Case Behavior**

   * **Non-callable** passed to `subscribe` -> raise `TypeError("Observer must be callable or have update method")`.
   * **Unknown token** passed to `unsubscribe` -> no error.
   * **Observer exception** during `update` -> log `"error:<token>:<exception>"` and continue.
   * **Pause** suppresses notifications until `resume()`.
   * **Once/TTL expiry** always unsubscribe exactly once when due, logging `"unsubscribe:<token>"`.
   * **Dead observers** detected via `weakref`; log `"dead:<token>"` then remove on next notify.

6. **Constraints**

   * **Thread safety**: use `threading.RLock` around all state mutations.
   * **Language**: Python 3.9+, standard library only.
   * **Style**: PEP-8 compliant (<=79 chars, proper imports, docstrings).
   * **Performance**: supports thousands of observers with efficient heap operations and lazy dead-entry purges.

7. **Important Notes**

   * You may assume callers pass valid floats for measurements no validation needed.
   * Logs and metrics need not be persisted beyond in-memory lists.
   * Avoid third-party dependencies.


In [ ]:
# code

import threading
import heapq
import itertools
import weakref
import time
import types

from typing import Callable, Dict, List, Optional, Tuple, Protocol, runtime_checkable


@runtime_checkable
class Observer(Protocol):
    def update(self, temp: float, humidity: float, pressure: float) -> None:
        ...


class _ObserverEntry:
    __slots__ = (
        "priority", "once", "ttl", "filter_fn",
        "fn_strong", "fn_ref", "token", "index",
    )

    def __init__(
        self,
        fn_strong: Optional[Callable],
        fn_ref: Optional[weakref.ReferenceType],
        priority: int,
        once: bool,
        ttl: Optional[int],
        filter_fn: Optional[Callable[[Tuple[float, float, float]], bool]],
        token: int,
        index: int,
    ) -> None:
        self.fn_strong = fn_strong
        self.fn_ref = fn_ref
        self.priority = priority
        self.once = once
        self.ttl = ttl
        self.filter_fn = filter_fn
        self.token = token
        self.index = index

    def __lt__(self, other) -> bool:
        return (self.priority, self.index) < (other.priority, other.index)


class WeatherData:
    _token_gen = itertools.count(1)

    def __init__(self) -> None:
        self._lock = threading.RLock()
        self._observers: List[_ObserverEntry] = []
        self._paused = False
        self._metrics: Dict[int, List[float]] = {}
        self._log: List[str] = []
        self._index_gen = itertools.count()
        self._data: Tuple[float, float, float] = (0.0, 0.0, 0.0)

    @staticmethod
    def _remove_dead(_ref: weakref.ReferenceType) -> None:
        # actual removal occurs in _cleanup_dead during notify
        pass

    def subscribe(
        self,
        callback: object,
        *,
        priority: int = 100,
        once: bool = False,
        ttl: Optional[int] = None,
        filter_fn: Optional[Callable[[Tuple[float, float, float]], bool]] = None,
    ) -> int:
        if not (callable(callback) or hasattr(callback, "update")):
            raise TypeError("Observer must be callable or have update method")

        fn_strong = None
        fn_ref = None
        if isinstance(callback, types.FunctionType):
            fn_strong = callback
        elif isinstance(callback, types.MethodType):
            fn_ref = weakref.WeakMethod(callback)
        elif hasattr(callback, "update"):
            fn_ref = weakref.ref(callback)
        else:
            fn_strong = callback

        token = next(WeatherData._token_gen)
        idx = next(self._index_gen)
        entry = _ObserverEntry(
            fn_strong, fn_ref, priority, once, ttl, filter_fn, token, idx
        )
        with self._lock:
            heapq.heappush(self._observers, entry)
            self._metrics[token] = []
            self._log.append(f"{time.perf_counter_ns()}:subscribe:{token}")
        return token

    def unsubscribe(self, token: int) -> None:
        with self._lock:
            before = len(self._observers)
            self._observers = [
                e for e in self._observers if e.token != token
            ]
            if len(self._observers) != before:
                heapq.heapify(self._observers)
                self._metrics.pop(token, None)
                self._log.append(f"{time.perf_counter_ns()}:unsubscribe:{token}")

    def pause(self) -> None:
        with self._lock:
            self._paused = True
            self._log.append(f"{time.perf_counter_ns()}:pause")

    def resume(self) -> None:
        with self._lock:
            self._paused = False
            self._log.append(f"{time.perf_counter_ns()}:resume")

    def set_measurements(self, temp: float, humidity: float, pressure: float) -> None:
        with self._lock:
            self._data = (temp, humidity, pressure)
            self._log.append(
                f"{time.perf_counter_ns()}:set:{temp},{humidity},{pressure}"
            )
        self._notify()

    def _cleanup_dead(self) -> None:
        alive: List[_ObserverEntry] = []
        for e in self._observers:
            fn = (
                e.fn_strong
                if e.fn_strong is not None
                else e.fn_ref() if e.fn_ref is not None
                else None
            )
            if fn is None:
                self._log.append(f"{time.perf_counter_ns()}:dead:{e.token}")
                self._metrics.pop(e.token, None)
            else:
                alive.append(e)
        if len(alive) != len(self._observers):
            self._observers = alive
            heapq.heapify(self._observers)

    def _notify(self) -> None:
        with self._lock:
            if self._paused:
                return
            self._cleanup_dead()
            snapshot = list(self._observers)

        snapshot.sort()
        temp, hum, pres = self._data

        for entry in snapshot:
            fn = (
                entry.fn_strong
                if entry.fn_strong is not None
                else entry.fn_ref() if entry.fn_ref is not None
                else None
            )
            if fn is None:
                continue

            call_fn = fn.update if hasattr(fn, "update") else fn
            if entry.filter_fn and not entry.filter_fn((temp, hum, pres)):
                continue

            start_ns = time.perf_counter_ns()
            try:
                call_fn(temp, hum, pres)
            except Exception as exc:
                self._log.append(
                    f"{time.perf_counter_ns()}:error:{entry.token}:{exc}"
                )
            end_ns = time.perf_counter_ns()
            duration = (end_ns - start_ns) / 1e9

            with self._lock:
                self._metrics.setdefault(entry.token, []).append(duration)
                self._log.append(
                    f"{time.perf_counter_ns()}:notify:"
                    f"{entry.token}:{duration}"
                )
                if entry.once:
                    self.unsubscribe(entry.token)
                elif entry.ttl is not None:
                    entry.ttl -= 1
                    if entry.ttl <= 0:
                        self.unsubscribe(entry.token)

    def get_metrics(self) -> Dict[int, float]:
        with self._lock:
            return {
                tok: sum(v) / len(v)
                for tok, v in self._metrics.items()
                if v
            }

    def get_log(self) -> List[str]:
        with self._lock:
            return list(self._log)


In [ ]:
# tests

import unittest
import threading
import gc
import weakref
import time

from main import WeatherData, _ObserverEntry

class TestWeatherDataObserverPattern(unittest.TestCase):
    def test_subscribe_non_callable(self):
        wd = WeatherData()
        with self.assertRaises(TypeError):
            wd.subscribe(123)

    def test_priority_ordering_and_ties(self):
        wd = WeatherData()
        calls = []
        # priority lower = earlier
        wd.subscribe(lambda t,h,p: calls.append('low'), priority=10)
        wd.subscribe(lambda t,h,p: calls.append('mid1'), priority=50)
        wd.subscribe(lambda t,h,p: calls.append('mid2'), priority=50)
        wd.subscribe(lambda t,h,p: calls.append('high'), priority=100)
        wd.set_measurements(1,1,1)
        self.assertEqual(calls, ['low','mid1','mid2','high'])

    def test_once_semantics_and_log(self):
        wd = WeatherData()
        calls = []
        token = wd.subscribe(lambda t,h,p: calls.append((t,h,p)), once=True)
        wd.set_measurements(2,2,2)
        wd.set_measurements(3,3,3)
        self.assertEqual(calls, [(2,2,2)])
        logs = wd.get_log()
        self.assertIn(f"unsubscribe:{token}", "".join(logs))

    def test_ttl_semantics_and_log(self):
        wd = WeatherData()
        calls = []
        token = wd.subscribe(lambda t,h,p: calls.append((t,h,p)), ttl=2)
        wd.set_measurements(10,10,10)
        wd.set_measurements(20,20,20)
        wd.set_measurements(30,30,30)
        self.assertEqual(calls, [(10,10,10),(20,20,20)])
        logs = wd.get_log()
        self.assertIn(f"unsubscribe:{token}", "".join(logs))

    def test_filter_fn(self):
        wd = WeatherData()
        calls = []
        wd.subscribe(lambda t,h,p: calls.append(p), filter_fn=lambda d: d[2]>50)
        wd.set_measurements(0,0,10)
        wd.set_measurements(0,0,60)
        self.assertEqual(calls, [60])

    def test_pause_resume(self):
        wd = WeatherData()
        calls = []
        wd.subscribe(lambda t,h,p: calls.append(p))
        wd.pause()
        wd.set_measurements(5,5,5)
        self.assertEqual(calls, [])
        wd.resume()
        # resume should NOT auto-notify per P/R
        self.assertEqual(calls, [])
        wd.set_measurements(6,6,6)
        self.assertEqual(calls, [6])

    def test_dead_observer_cleanup(self):
        wd = WeatherData()
        class Obs:
            def __init__(self):
                self.called = False
            def update(self, t,h,p):
                self.called = True
        obs = Obs()
        token = wd.subscribe(obs)
        ref = weakref.ref(obs)
        del obs
        gc.collect()
        wd.set_measurements(1,1,1)
        logs = wd.get_log()
        self.assertIn(f"dead:{token}", "".join(logs))
        # no exception

    def test_exception_isolation(self):
        wd = WeatherData()
        called = []
        wd.subscribe(lambda t,h,p: (_ for _ in ()).throw(ValueError("err")))
        wd.subscribe(lambda t,h,p: called.append(True))
        wd.set_measurements(1,1,1)
        self.assertEqual(called,[True])
        logs = wd.get_log()
        self.assertTrue(any("error:" in e for e in logs))

    def test_get_metrics(self):
        wd = WeatherData()
        wd.subscribe(lambda t,h,p: None)
        wd.set_measurements(1,1,1)
        m = wd.get_metrics()
        self.assertTrue(isinstance(m, dict))
        # average > 0
        self.assertTrue(all(v>=0 for v in m.values()))

    def test_unsubscribe_no_error(self):
        wd = WeatherData()
        wd.unsubscribe(999)
        # no exception, no log for unknown token
        logs = wd.get_log()
        self.assertNotIn("unsubscribe:999", "".join(logs))

    def test_concurrent_operations(self):
        wd = WeatherData()
        stop = threading.Event()
        errors = []
        def spam():
            while not stop.is_set():
                try:
                    wd.set_measurements(1,2,3)
                    wd.subscribe(lambda t,h,p: None)
                except Exception as e:
                    errors.append(e)
        t = threading.Thread(target=spam)
        t.start()
        time.sleep(0.1)
        stop.set()
        t.join()
        self.assertFalse(errors)

if __name__ == '__main__':
    unittest.main()




# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/3172b280-bba1-449e-9d0a-944f2815ca2f?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67967%2Fview

#### Model code:

```python
import threading
import heapq
import weakref
import time
from typing import Callable, List, Optional, Tuple, Dict
from datetime import datetime

class _ObserverEntry:
    def __init__(self, token, priority, callback_ref, once, ttl, filter_fn, index):
        self.token = token
        self.priority = priority
        self.ref = callback_ref
        self.once = once
        self.ttl = ttl
        self.filter_fn = filter_fn
        self.index = index
        
    def __lt__(self, other):
        return (self.priority, self.index) < (other.priority, other.index)


class WeatherData:
    def __init__(self):
        self._observers = []
        self._paused = False
        self._lock = threading.RLock()
        self._token_counter = 0
        self._insertion_index = 0
        self._log = []
        self._metrics = {}
        self._measurements = None
    
    def subscribe(self, callback: Callable[[float, float, float], None], *,
                  priority: int = 100, once: bool = False, ttl: Optional[int] = None,
                  filter_fn: Optional[Callable[[Tuple[float, float, float]], bool]] = None
                 ) -> int:
        if not callable(callback) and not hasattr(callback, 'update'):
            raise TypeError("Observer must be callable or have update method")
        
        with self._lock:
            self._token_counter += 1
            token = self._token_counter
            callback_ref = weakref.ref(callback)
            entry = _ObserverEntry(
                token, priority, callback_ref,
                once, ttl, filter_fn,
                self._insertion_index
            )
            heapq.heappush(self._observers, entry)
            self._insertion_index += 1
            self._metrics[token] = []
            self._log_event(f"subscribe:{token}")
            return token

    def unsubscribe(self, token: int) -> None:
        with self._lock:
            self._observers = [
                obs for obs in self._observers
                if obs.token != token
            ]
            heapq.heapify(self._observers)
            self._log_event(f"unsubscribe:{token}")

    def pause(self) -> None:
        with self._lock:
            self._paused = True
            self._log_event("pause")

    def resume(self) -> None:
        with self._lock:
            self._paused = False
            self._log_event("resume")
            if self._measurements:
                self._notify_observers(*self._measurements)
                # <Issue>: Spec does not require auto-notify on resume

    def set_measurements(self, temp: float, humidity: float, pressure: float) -> None:
        measurements = (temp, humidity, pressure)
        with self._lock:
            self._measurements = measurements
            self._log_event(f"set_measurements:{measurements}")
            if not self._paused:
                self._notify_observers(temp, humidity, pressure)

    def _notify_observers(self, temp: float, humidity: float, pressure: float) -> None:
        now = time.time()
        to_remove = []

        # <Issue>: No lock held during iteration over _observers
        for entry in sorted(self._observers):
            callback = entry.ref()
            if callback is None:
                self._log_event(f"dead:{entry.token}")
                to_remove.append(entry)
                continue

            if entry.filter_fn and not entry.filter_fn((temp, humidity, pressure)):
                continue
            
            try:
                if hasattr(callback, 'update'):
                    callback.update(temp, humidity, pressure)
                else:
                    callback(temp, humidity, pressure)
                
                elapsed = time.time() - now
                # <Issue>: 'elapsed' includes cumulative time from start, not per-callback
                self._metrics[entry.token].append(elapsed)

                if entry.once or (entry.ttl is not None and entry.ttl <= 1):
                    to_remove.append(entry)
                    self._log_event(f"unsubscribe:{entry.token}")
                elif entry.ttl is not None:
                    entry.ttl -= 1
            except Exception as ex:
                self._log_event(f"error:{entry.token}:{ex}")
        
        for entry in to_remove:
            self.unsubscribe(entry.token)

    def get_metrics(self) -> Dict[int, float]:
        with self._lock:
            return {
                token: (sum(times) / len(times)) if times else 0.0
                for token, times in self._metrics.items()
            }

    def get_log(self) -> List[str]:
        with self._lock:
            return list(self._log)

    def _log_event(self, event: str) -> None:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        # <Issue>: Uses datetime.now(); spec requires nanosecond-precision perf_counter_ns()
        self._log.append(f"{timestamp} {event}")

```